In [58]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
import rubrix as rb
from rubrix.labeling.text_classification.weak_labels import  WeakLabels
from rubrix.labeling.text_classification import MajorityVoter
from rubrix.labeling.text_classification import Snorkel, FlyingSquid

In [2]:
df = pd.read_excel('Labelled.xlsx')

In [3]:
df.shape

(9400, 2)

In [10]:
train_data = df[df['label'].isnull()]
test_data  = df[df['label'].notnull()]
#df.shape

In [23]:
#train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
train_data

,text,label
3080,ആയിട്ട് Compare ചെയ്യാൻ പറ്റിയ ഐറ്റം നെരസിംഹം ...,NaN
3156,ഈ മൈരൻ പറയുന്നത് ഒന്നും വിശ്വസിക്കാൻ പറ്റില്ല,NaN
3648,ആളെക്കൂട്ടാൻ തന്ത എത്രയാണെന്ന് ചോദിച്ചാൽ പത്തു...,NaN
5020,തായോളി.. എന്ന് നിന്നെ വിളിക്കുന്നത് കുറവാണ്.....,NaN
5021,അങ്ങനെ ട്വിറ്റെറിലേക്ക് ഒരു മൈരൻ കൂടി വന്നിരിക...,NaN
...,...,...
9395,പേടിച്ച് അക്കൗണ്ട് പൂട്ടിക്കാൻ നടന്ന ആ സൊച്യലി...,NaN
9396,നീ എന്തൊരു മൈരൻ ആണ് 😌,NaN
9397,വാരിയം കുണ്ടൻ രായപ്പനും കാമ്പയിനിൽ ചോദ്യം ചെ...,NaN
9398,ജീ കണ്ടാ നാറികൾ ക്കു വേണ്ടി എന്തിനാ താങ്കൾ ത...,NaN


In [14]:
records = [rb.TextClassificationRecord(text=row.text)
    for i,row in train_data.iterrows()
            ]

# build records from the test dataset with annotation
labels = ['Offensive-Ind', 'Offensive-Untargetted', 'Offensive-Group', 'Not-Offensive']
records += [rb.TextClassificationRecord(text=row.text,annotation=row.label)
    for i,row in test_data.iterrows()
        ]

In [15]:
len(records)

9400

In [16]:
rb.log(records, name="weak_supervision")

  0%|          | 0/9400 [00:00<?, ?it/s]

9400 records logged to http://localhost:6900/datasets/rubrix/weak_supervision


BulkResponse(dataset='weak_supervision', processed=9400, failed=0)

In [17]:
weak_labels = WeakLabels(dataset="weak_supervision")

Preparing rules:   0%|          | 0/19 [00:00<?, ?it/s]

Applying rules:   0%|          | 0/9400 [00:00<?, ?it/s]

In [18]:
majority_model = MajorityVoter(weak_labels)

In [19]:
print(majority_model.score(tie_break_policy="random",output_str=True))

                       precision    recall  f1-score   support

      Offensive-Group       0.35      0.75      0.48       639
        Offensive-Ind       0.83      0.55      0.66      3603
Offensive-Untargetted       0.25      0.42      0.31       443
        Not-Offensive       0.10      0.16      0.13       332

             accuracy                           0.54      5017
            macro avg       0.38      0.47      0.40      5017
         weighted avg       0.67      0.54      0.57      5017



In [20]:
# To get the weak labels of the training data based on majority voting
records_for_training = majority_model.predict()

In [53]:
snorkel_model = Snorkel(weak_labels)
snorkel_model.fit(lr=0.001, n_epochs=100)

100%|██████████| 100/100 [00:00<00:00, 880.00epoch/s]


In [54]:
print(snorkel_model.score(output_str=True))

                       precision    recall  f1-score   support

      Offensive-Group       0.57      0.75      0.65       589
        Offensive-Ind       0.83      0.81      0.82      1972
Offensive-Untargetted       0.57      0.58      0.58       245
        Not-Offensive       0.21      0.02      0.03       165

             accuracy                           0.74      2971
            macro avg       0.55      0.54      0.52      2971
         weighted avg       0.72      0.74      0.72      2971



In [56]:
records_for_training = snorkel_model.predict()

# optional: log the records to a new dataset in Rubrix
rb.log(records_for_training, name="snorkel_results")

# extract training data
training_data = pd.DataFrame(
    [
        {"text": rec.text, "label": rec.prediction[0][0]}
        for rec in records_for_training
    ]
)

  0%|          | 0/2705 [00:00<?, ?it/s]

2705 records logged to http://localhost:6900/datasets/rubrix/snorkel_results


In [59]:
flyingsquid_model = FlyingSquid(weak_labels)

# we fit the model
flyingsquid_model.fit()

/home/sudheesh/anaconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(


In [60]:
print(flyingsquid_model.score(output_str=True))

                       precision    recall  f1-score   support

      Offensive-Group       0.58      0.74      0.65       574
        Offensive-Ind       0.83      0.83      0.83      1935
Offensive-Untargetted       0.58      0.58      0.58       232
        Not-Offensive       0.21      0.02      0.03       161

             accuracy                           0.74      2902
            macro avg       0.55      0.54      0.52      2902
         weighted avg       0.73      0.74      0.73      2902

